## Part- 1 (Problem Statement)

### Where in London would it be best to open a restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another?

### The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a restaurant in London and wants to know if it is a viable option where they intend on opening.

## Part- 2 Data
### The data that will be used is Foursquare data for London city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

In [2]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 962kB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 41.1MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 7.6MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 28.6MB/s eta 0:00:01█████████████      | 8.9MB 28.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 22.0MB/s eta 0:00:01
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
     |████████████████████████████████| 112kB 5.6MB/s eta 0:00:01


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
address = 'London, London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London is 51.5073219, -0.1276474.


In [5]:
CLIENT_ID = '5ROCVQZ3VOHGPZUX1ANPLAMEGIYYZZM1BCV0KUG5S0FKX3Q1' # your Foursquare ID
CLIENT_SECRET = '5OAZIN50SLEDEEXOD144NINCLENWXWM3QNFSTFUOA121I3ND' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ROCVQZ3VOHGPZUX1ANPLAMEGIYYZZM1BCV0KUG5S0FKX3Q1
CLIENT_SECRET:5OAZIN50SLEDEEXOD144NINCLENWXWM3QNFSTFUOA121I3ND


In [6]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5ROCVQZ3VOHGPZUX1ANPLAMEGIYYZZM1BCV0KUG5S0FKX3Q1&client_secret=5OAZIN50SLEDEEXOD144NINCLENWXWM3QNFSTFUOA121I3ND&ll=51.5073219,-0.1276474&v=20180605&query=Restaurant&radius=50000&limit=100'

In [8]:
results = requests.get(url).json()

In [9]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4af41e26f964a52024f021e3,Portrait Restaurant,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1593967298,False,St. Martin's Pl,Trafalgar Sq,51.509259,-0.128202,"[{'label': 'display', 'lat': 51.50925927272873...",219,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin's Pl (Trafalgar Sq), London, Great...",130427500,NaN
1,5001b8e6e4b05e79b0b3b84d,Bill's Restaurant,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",v-1593967298,False,36-44 Brewer St,Lexington St,51.511834,-0.135080,"[{'label': 'display', 'lat': 51.51183377093123...",719,W1F 9TB,GB,London,Greater London,United Kingdom,"[36-44 Brewer St (Lexington St), Soho, Greater...",NaN,NaN
2,4b483c31f964a520754a26e3,Kowloon Restaurant and Cakeshop,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1593967298,False,21-22 Gerrard St,NaN,51.511503,-0.131904,"[{'label': 'display', 'lat': 51.51150273700102...",550,W1D 6JH,GB,Chinatown,Greater London,United Kingdom,"[21-22 Gerrard St, Chinatown, Greater London, ...",NaN,NaN
3,4ed23e3b2c5bc35059276f87,London Chinatown Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1593967298,False,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",547,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,NaN
4,4c39bcafedba9521cf6cd625,New Loon Fung Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1593967298,False,42-44 Gerrard St,NaN,51.512039,-0.130371,"[{'label': 'display', 'lat': 51.51203891754673...",557,W1D 5QG,GB,London,Greater London,United Kingdom,"[42-44 Gerrard St, London, Greater London, W1D...",NaN,NaN


In [10]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Portrait Restaurant,Café,St. Martin's Pl,Trafalgar Sq,51.509259,-0.128202,"[{'label': 'display', 'lat': 51.50925927272873...",219,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin's Pl (Trafalgar Sq), London, Great...",NaN,4af41e26f964a52024f021e3
1,Bill's Restaurant,English Restaurant,36-44 Brewer St,Lexington St,51.511834,-0.135080,"[{'label': 'display', 'lat': 51.51183377093123...",719,W1F 9TB,GB,London,Greater London,United Kingdom,"[36-44 Brewer St (Lexington St), Soho, Greater...",NaN,5001b8e6e4b05e79b0b3b84d
2,Kowloon Restaurant and Cakeshop,Chinese Restaurant,21-22 Gerrard St,NaN,51.511503,-0.131904,"[{'label': 'display', 'lat': 51.51150273700102...",550,W1D 6JH,GB,Chinatown,Greater London,United Kingdom,"[21-22 Gerrard St, Chinatown, Greater London, ...",NaN,4b483c31f964a520754a26e3
3,London Chinatown Restaurant,Chinese Restaurant,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",547,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,4ed23e3b2c5bc35059276f87
4,New Loon Fung Chinese Restaurant,Chinese Restaurant,42-44 Gerrard St,NaN,51.512039,-0.130371,"[{'label': 'display', 'lat': 51.51203891754673...",557,W1D 5QG,GB,London,Greater London,United Kingdom,"[42-44 Gerrard St, London, Greater London, W1D...",NaN,4c39bcafedba9521cf6cd625


In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map